In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install optuna

In [4]:
import pandas as pd
import optuna
optuna.logging.set_verbosity(optuna.logging.ERROR)
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_predict, train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score
import logging
import pickle
from sklearn.metrics import classification_report

In [5]:
df = pd.read_csv("/content/drive/MyDrive/train_data_attrition_scaling.csv")

In [6]:
# Đọc dữ liệu và tách features - target
X = df.drop(columns=['Attrition'])
y = df['Attrition']

In [7]:
# Tách tập train và test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [8]:
# Cấu hình logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [9]:
# Hàm tối ưu hóa
def objective_rf(trial):
    try:
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300),
            'max_depth': trial.suggest_int('max_depth', 3, 20),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
            'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy']),
            'class_weight': 'balanced',  # để xử lý imbalance
            'random_state': 42
        }

        model = RandomForestClassifier(**params)
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1')

        f1_mean = cv_scores.mean()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:, 1]

        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        trial.set_user_attr('recall', recall)
        trial.set_user_attr('precision', precision)
        trial.set_user_attr('roc_auc', roc_auc)

        logger.info(f"Trial {trial.number}: F1={f1_mean:.4f}, Recall={recall:.4f}, Precision={precision:.4f}, ROC AUC={roc_auc:.4f}")
        return f1_mean

    except Exception as e:
        logger.error(f"Trial {trial.number} failed: {str(e)}")
        return 0.0

In [11]:
# Tối ưu hóa
study = optuna.create_study(direction='maximize')
study.optimize(objective_rf, n_trials=100)

In [17]:
# Lấy trial tốt nhất
best_trial = study.best_trial
print("\nTham số tốt nhất cho Random Forest:")
print(best_trial.params)
print(f" F1 Score cross-validated tốt nhất: {best_trial.value:.4f}")
print(f" Recall trên tập test: {best_trial.user_attrs['recall']:.4f}")
print(f" Precision trên tập test: {best_trial.user_attrs['precision']:.4f}")
print(f" ROC AUC trên tập test: {best_trial.user_attrs['roc_auc']:.4f}")


Tham số tốt nhất cho Random Forest:
{'n_estimators': 289, 'max_depth': 18, 'min_samples_split': 2, 'max_features': 'sqrt', 'criterion': 'gini'}
 F1 Score cross-validated tốt nhất: 0.9091
 Recall trên tập test: 0.8814
 Precision trên tập test: 0.8927
 ROC AUC trên tập test: 0.9508


In [22]:
# Train lại với toàn bộ tập huấn luyện
final_model = RandomForestClassifier(**best_trial.params, class_weight='balanced', random_state=42)
final_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=18, n_estimators=289,
                       random_state=42)

In [23]:
# Đánh giá cuối cùng trên tập kiểm tra
y_pred = final_model.predict(X_test)
y_proba = final_model.predict_proba(X_test)[:, 1]
print("\nĐánh giá cuối cùng trên tập kiểm tra:")
print(f"F1 Score: {f1_score(y_test, y_pred):.4f}")
print(f"Recall: {recall_score(y_test, y_pred):.4f}")
print(f"Precision: {precision_score(y_test, y_pred):.4f}")
print(f"ROC AUC: {roc_auc_score(y_test, y_proba):.4f}")

print("\nBáo cáo phân loại chi tiết:")
print(classification_report(y_test, y_pred, digits=3))


Đánh giá cuối cùng trên tập kiểm tra:
F1 Score: 0.8870
Recall: 0.8814
Precision: 0.8927
ROC AUC: 0.9508

Báo cáo phân loại chi tiết:
              precision    recall  f1-score   support

           0      0.883     0.895     0.889       237
           1      0.893     0.881     0.887       236

    accuracy                          0.888       473
   macro avg      0.888     0.888     0.888       473
weighted avg      0.888     0.888     0.888       473



In [26]:
# Sau khi có best_params từ Optuna

# 1. Huấn luyện trên TOÀN BỘ tập train gốc
final_model = RandomForestClassifier(
    **best_trial.params,
    class_weight='balanced',
    random_state=42
)
final_model.fit(X, y)  # Sử dụng toàn bộ dữ liệu X, y từ file gốc

# 2. Lưu mô hình
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(final_model, f)
print("\nMô hình đã được lưu vào 'random_forest_model.pkl'")


Mô hình đã được lưu vào 'random_forest_model.pkl'
